In [5]:
import pandas as pd
import numpy as np
import ETL.config as config
import ETL.utils as utils
import ETL.extract_data as extract
import ETL.load_data as load
import logging
import re
import importlib
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
importlib.reload(config)
importlib.reload(extract)
logger = logging.getLogger(__name__)

In [18]:
columns_to_extract = ['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip', 'merchant_id', 'errors']
df = extract.extract_data(config.FILTERED_TRANSACTIONS_FILE_LOCAL,'TRANSACTIONS')

In [4]:
df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,18881487,2017-01-01 00:01:00,301,3742,$7.05,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
1,18881488,2017-01-01 00:01:00,1129,2677,$100.00,Chip Transaction,27092,San Marcos,TX,78666.0,4829,NaN
2,18881490,2017-01-01 00:04:00,316,5857,$130.29,Chip Transaction,6389,Houston,TX,77047.0,5300,NaN
3,18881491,2017-01-01 00:04:00,566,3439,$12.39,Online Transaction,18563,ONLINE,NaN,NaN,4121,NaN
4,18881492,2017-01-01 00:04:00,1357,3649,$20.67,Chip Transaction,61195,Tacoma,WA,98404.0,5541,NaN


In [75]:
ids = [19100281, 19110974, 19118596, 19220791, 19222235, 19245702, 19335469, 19353887, 19363476, 19467826, 19501046, 19638179, 19661597, 19771623, 19863784, 20207021, 20234388, 20239423, 20280867, 20295484, 20374879, 20635830, 20674353, 20739797, 20758120, 22144017, 22144852, 22148139, 22157714, 22159060, 22949891, 23325365, 23340160]
df.loc[df['id'].isin(ids)].groupby(['merchant_id','merchant_city','merchant_state','zip'])['id'].count().reset_index()
# print(df.loc[df['id'].isin(ids)]['merchant_id'].value_counts())
# print(df.loc[df['id'].isin(ids)]['merchant_city'].value_counts())
# print(df.loc[df['id'].isin(ids)]['merchant_state'].value_counts())
# print(df.loc[df['id'].isin(ids)]['zip'].value_counts())
# print(df.loc[df['id'].isin(ids)]['mcc'].value_counts())

,merchant_id,merchant_city,merchant_state,zip,id
0,34490,Bell Gardens,CA,90201.0,8
1,34490,Panama City,FL,32408.0,20


In [53]:
df['id'].duplicated().sum()

np.int64(0)

In [5]:
df.loc[df['merchant_id'].isin([75506]),][['merchant_id','merchant_state','merchant_city','zip', 'mcc']].drop_duplicates()


,merchant_id,merchant_state,merchant_city,zip,mcc
14517,75506,TX,Carrollton,75006.0,5921
21916,75506,MI,Holly,48442.0,5921
3887438,75506,TX,Bryan,77802.0,5921
3912147,75506,AL,Owens Cross Roads,35763.0,5921


In [6]:
df.duplicated().sum()


np.int64(0)

In [7]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954066 entries, 0 to 3954065
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              3954066 non-null  int64  
 1   date            3954066 non-null  object 
 2   client_id       3954066 non-null  int64  
 3   card_id         3954066 non-null  int64  
 4   amount          3954066 non-null  object 
 5   use_chip        3954066 non-null  object 
 6   merchant_id     3954066 non-null  int64  
 7   merchant_city   3954066 non-null  object 
 8   merchant_state  3473293 non-null  object 
 9   zip             3444171 non-null  float64
 10  mcc             3954066 non-null  int64  
 11  errors          62903 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 362.0+ MB


In [8]:
df.describe(include= 'all')

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
count,3.954066e+06,3954066,3.954066e+06,3.954066e+06,3954066,3954066,3.954066e+06,3954066,3473293,3.444171e+06,3.954066e+06,62903
unique,NaN,1201258,NaN,NaN,52701,3,NaN,9870,147,NaN,NaN,21
top,NaN,2018-11-23 07:16:00,NaN,NaN,$80.00,Chip Transaction,NaN,ONLINE,CA,NaN,NaN,Insufficient Balance
freq,NaN,17,NaN,NaN,38607,2809950,NaN,480773,423235,NaN,NaN,38838
mean,2.131893e+07,NaN,1.026916e+03,3.485414e+03,NaN,NaN,4.766662e+04,NaN,NaN,5.132725e+04,5.564363e+03,NaN
std,1.409082e+06,NaN,5.805413e+02,1.676195e+03,NaN,NaN,2.582575e+04,NaN,NaN,2.941566e+04,8.777942e+02,NaN
min,1.888149e+07,NaN,0.000000e+00,0.000000e+00,NaN,NaN,2.000000e+00,NaN,NaN,1.001000e+03,1.711000e+03,NaN
25%,2.009921e+07,NaN,5.250000e+02,2.416000e+03,NaN,NaN,2.587500e+04,NaN,NaN,2.858600e+04,5.300000e+03,NaN
50%,2.131673e+07,NaN,1.069000e+03,3.596000e+03,NaN,NaN,4.592600e+04,NaN,NaN,4.780300e+04,5.499000e+03,NaN
75%,2.253873e+07,NaN,1.531000e+03,4.894000e+03,NaN,NaN,6.759100e+04,NaN,NaN,7.801600e+04,5.812000e+03,NaN


In [9]:
df['id'].duplicated().sum()

np.int64(0)

In [10]:
df['use_chip'].value_counts(dropna=False)

use_chip
Chip Transaction      2809950
Swipe Transaction      666777
Online Transaction     477339
Name: count, dtype: int64

In [11]:
df['errors'].value_counts(dropna=False)

errors
NaN                                      3891163
Insufficient Balance                       38838
Bad PIN                                     9601
Technical Glitch                            7828
Bad Card Number                             2389
Bad Expiration                              1859
Bad CVV                                     1783
Bad Zipcode                                  328
Bad PIN,Insufficient Balance                  92
Insufficient Balance,Technical Glitch         72
Bad PIN,Technical Glitch                      24
Bad Card Number,Insufficient Balance          21
Bad CVV,Insufficient Balance                  14
Bad Expiration,Bad CVV                        13
Bad Card Number,Bad CVV                       13
Bad Expiration,Insufficient Balance            8
Bad Expiration,Technical Glitch                7
Bad Card Number,Bad Expiration                 5
Bad Zipcode,Insufficient Balance               3
Bad Card Number,Technical Glitch               3
Bad CVV,Techn

In [12]:
df.loc[(df['errors'] == 'Bad PIN,Insufficient Balance') | (df['errors'] == 'Insufficient Balance,Technical Glitch')]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
41785,18932850,2017-01-11 20:48:00,1066,1301,$100.00,Chip Transaction,27092,Newtown,PA,18940.0,4829,"Bad PIN,Insufficient Balance"
70900,18968680,2017-01-19 12:48:00,742,3943,$122.00,Swipe Transaction,59935,Manassas,VA,20109.0,5499,"Bad PIN,Insufficient Balance"
73792,18972236,2017-01-20 08:48:00,1209,4573,$112.35,Chip Transaction,50867,Yonkers,NY,10710.0,5541,"Insufficient Balance,Technical Glitch"
87704,18989416,2017-01-24 00:44:00,1357,4611,$22.22,Chip Transaction,50783,Tacoma,WA,98404.0,5411,"Bad PIN,Insufficient Balance"
132631,19044741,2017-02-04 14:57:00,1425,5929,$37.14,Swipe Transaction,83743,Essex,IL,60935.0,7538,"Bad PIN,Insufficient Balance"
...,...,...,...,...,...,...,...,...,...,...,...,...
3865160,23652131,2019-10-08 13:50:00,779,119,$71.11,Chip Transaction,20561,Pottsboro,TX,75076.0,5912,"Bad PIN,Insufficient Balance"
3876947,23666767,2019-10-11 15:30:00,148,2144,$85.59,Chip Transaction,20561,Canterbury,NH,3224.0,5912,"Bad PIN,Insufficient Balance"
3877998,23668052,2019-10-11 22:21:00,1123,3010,$61.59,Chip Transaction,20416,Laurel Hill,NC,28351.0,5712,"Bad PIN,Insufficient Balance"
3909226,23706600,2019-10-20 07:41:00,674,4786,$60.55,Chip Transaction,37369,San Diego,CA,92154.0,4111,"Insufficient Balance,Technical Glitch"


In [13]:
df['amount'].str.replace(r'[A-Za-z0-9]','',regex=True).value_counts()

amount
$.     3760589
$-.     193477
Name: count, dtype: int64

In [14]:
engine = load.create_database_connection()
query = 'SELECT * FROM financial.dim_merchant'
mcc = pd.read_sql(query,engine)
mcc.head(10)

,merchant_key,merchant_id,merchant_state,merchant_city,zip,mcc,mcc_description,load_timestamp
0,0,39021,ONLINE,ONLINE,0,4784,Tolls and Bridge Fees,2025-07-09 11:23:14.460641
1,1,27092,TX,San Marcos,78666,4829,Money Transfer,2025-07-09 11:23:14.464539
2,2,6389,TX,Houston,77047,5300,Wholesale Clubs,2025-07-09 11:23:14.464539
3,3,18563,ONLINE,ONLINE,0,4121,Taxicabs and Limousines,2025-07-09 11:23:14.464539
4,4,61195,WA,Tacoma,98404,5541,Service Stations,2025-07-09 11:23:14.464539
5,5,58215,OH,Cleveland,44118,5813,Drinking Places (Alcoholic Beverages),2025-07-09 11:23:14.464539
6,6,75781,NY,Brooklyn,11210,5411,"Grocery Stores, Supermarkets",2025-07-09 11:23:14.464539
7,8,73186,ONLINE,ONLINE,0,4814,Telecommunication Services,2025-07-09 11:23:14.464539
8,9,27092,IN,Crown Point,46307,4829,Money Transfer,2025-07-09 11:23:14.464539
9,10,85986,NY,Jamaica,11434,5812,Eating Places and Restaurants,2025-07-09 11:23:14.464539


In [15]:
test_mcc= mcc.drop(columns='merchant_key')
print(test_mcc.duplicated().sum())
test_mcc[test_mcc['merchant_id'] == 27092]

0


,merchant_id,merchant_state,merchant_city,zip,mcc,mcc_description,load_timestamp
1,27092,TX,San Marcos,78666,4829,Money Transfer,2025-07-09 11:23:14.464539
8,27092,IN,Crown Point,46307,4829,Money Transfer,2025-07-09 11:23:14.464539
54,27092,AZ,Pinetop,85935,4829,Money Transfer,2025-07-09 11:23:14.465664
148,27092,FL,Gainesville,32608,4829,Money Transfer,2025-07-09 11:23:14.468758
149,27092,WI,Suring,54174,4829,Money Transfer,2025-07-09 11:23:14.468758
...,...,...,...,...,...,...,...
119699,27092,WI,Coon Valley,54623,4829,Money Transfer,2025-07-09 11:23:32.040131
119855,27092,IN,Bloomington,47401,4829,Money Transfer,2025-07-09 11:23:32.045374
121320,27092,TX,Dallas,75220,4829,Money Transfer,2025-07-09 11:23:32.315355
121466,27092,CA,Chino Hills,91709,4829,Money Transfer,2025-07-09 11:23:32.319505


In [ ]:
def transform_fact_transactions(df: pd.DataFrame, data_name: str)->pd.DataFrame: 
    '''
    Function to transform TRANSACTIONS fact data

    Parameters:
        df (pd.DataFrame): The DataFrame to transform.
        data_name (str): data name of the dim
    Returns:
        pd.DataFrame: The transformed DataFrame.
    
    '''
    
    columns_to_rename = {'id': 'transaction_id' , 'use_chip': 'card_entry_method', 'errors':'transaction_error'
                         ,'date':'transaction_date', 'client_id':'user_id'}
    query = 'SELECT merchant_key,merchant_id,merchant_city,merchant_state,zip,mcc FROM financial.dim_merchant'
    merge_on = ['merchant_id','merchant_city','merchant_state', 'zip','mcc']
    columns_to_remove = ['merchant_id', 'merchant_city', 'merchant_state','zip', 'mcc']
    bins   = [-1, 15, 30, 45, 59] #use it to create ['minute_block']
    labels = ['0015', '1630', '3145', '4659'] #use it to create ['minute_block']

    try:
        logger.info(f'Starting transform {data_name} fact data')

        df = df.copy()
        df_cleaned = df.rename(columns=columns_to_rename)
        df_cleaned['amount'] = df_cleaned['amount'].str.replace('$','').astype(float)
        df_cleaned.loc[(df_cleaned['merchant_state'].isna()) & (df_cleaned['merchant_city'] == 'ONLINE'), ['merchant_state']] = 'ONLINE'
        df_cleaned.loc[(df_cleaned['merchant_state'] == 'ONLINE') & (df_cleaned['merchant_city'] == 'ONLINE'), 'zip'] = 0
        df_cleaned['zip'] = df_cleaned['zip'].fillna(0).astype(int)
        df_cleaned['transaction_date'] = pd.to_datetime(df_cleaned['transaction_date'])
        #df_cleaned['zip'] = df_cleaned['zip'].astype(str).str.replace(r'\.0$', '', regex=True)
        df['minute_block'] = pd.cut(pd.to_datetime(df['date']).dt.minute, bins=bins, labels=labels).astype(str)
        df_cleaned['date_key'] = (
            df_cleaned['date'].astype(str)
                .str.replace(r'[^A-Za-z0-9]', '', regex=True)
                .str[:8]
            + df_cleaned['minute_block'].astype(str)
            )

        

        engine = load.create_database_connection()
        mcc = pd.read_sql(query,engine)
        df_cleaned = df_cleaned.merge(mcc, left_on= merge_on, right_on= merge_on, how='left')
        df_cleaned = df_cleaned.drop(columns= columns_to_remove)

    
        logger.info(f'✅ Successfully transformed {data_name} data with {len(df_cleaned)} records')
        return df_cleaned
        

    except Exception as e:
        logger.info(f'❌Error transforming {data_name} data: {e}')
        raise

In [87]:
test_copy = df.copy()
transactions = transform_fact_transactions(test_copy,'TRANSACTIONS')
        

In [39]:
transactions.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954099 entries, 0 to 3954098
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   transaction_id     3954099 non-null  int64         
 1   transaction_date   3954099 non-null  datetime64[ns]
 2   user_id            3954099 non-null  int64         
 3   card_id            3954099 non-null  int64         
 4   amount             3954099 non-null  float64       
 5   card_entry_method  3954099 non-null  object        
 6   transaction_error  62903 non-null    object        
 7   merchant_key       3954099 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 241.3+ MB


In [88]:
transactions['transaction_id'].duplicated().sum()

np.int64(0)

In [40]:
transactions.sample(10)

,transaction_id,transaction_date,user_id,card_id,amount,card_entry_method,transaction_error,merchant_key
2356406,21785227,2018-09-08 08:04:00,116,6072,55.00,Chip Transaction,NaN,289
940106,20039640,2017-09-03 08:32:00,379,327,799.55,Chip Transaction,NaN,940091
3014597,22599660,2019-02-27 22:12:00,719,5518,64.78,Chip Transaction,NaN,755171
1683928,20954900,2018-03-16 10:31:00,1966,6029,29.78,Chip Transaction,NaN,1401
2234669,21635225,2018-08-07 14:22:00,28,5198,87.47,Swipe Transaction,NaN,242297
2307228,21724751,2018-08-26 10:46:00,1861,4898,71.00,Chip Transaction,NaN,26699
2886321,22440477,2019-01-25 07:49:00,882,3467,27.37,Online Transaction,NaN,10467
3015719,22601028,2019-02-28 08:56:00,1946,5701,22.70,Chip Transaction,NaN,141845
2927151,22491005,2019-02-04 16:52:00,1301,3189,0.65,Chip Transaction,NaN,10620
8459,18891909,2017-01-03 08:50:00,921,3135,75.49,Swipe Transaction,NaN,1777


In [89]:
transactions['merchant_key'].nunique(dropna=False)

121992

In [36]:
transactions['transaction_id'].astype(str).str.len().value_counts()

transaction_id
8    3954099
Name: count, dtype: int64

In [ ]:
print(transactions[transactions['transaction_id'].duplicated()].sort_values(by='transaction_id')['transaction_id'].to_list())

[19100281, 19110974, 19118596, 19220791, 19222235, 19245702, 19335469, 19353887, 19363476, 19467826, 19501046, 19638179, 19661597, 19771623, 19863784, 20207021, 20234388, 20239423, 20280867, 20295484, 20374879, 20635830, 20674353, 20739797, 20758120, 22144017, 22144852, 22148139, 22157714, 22159060, 22949891, 23325365, 23340160]


In [81]:
print(transactions.loc[transactions['transaction_id'].isin(ids)]['merchant_key'].to_list())

[177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 797686, 1455754, 797686, 1455754, 177861, 186579, 797686, 1455754, 177861, 186579, 177861, 186579, 177861, 186579, 177861, 186579, 797686, 1455754, 797686, 1455754, 177861, 186579, 2646670, 2647344, 2646670, 2647344, 2646670, 2647344, 2646670, 2647344, 2646670, 2647344, 797686, 1455754, 797686, 1455754, 797686, 1455754]


In [54]:
#transactions['card_used_method'].str.replace(r'(?i)transaction', '', regex=True).str.lower().str.strip()

In [91]:
#transactions.loc[transactions['zip'].isna()]

In [92]:
transactions['amount'].sum()

np.float64(168730853.87999982)

In [3]:
import pandas as pd
import ETL.config as config
import ETL.utils as utils
import ETL.load_data as load
import ETL.extract_data as extract
import logging


logger = logging.getLogger(__name__)

def transform_fact_transactions_(df: pd.DataFrame, data_name: str)->pd.DataFrame: 
    '''
    Function to transform TRANSACTIONS fact data

    Parameters:
        df (pd.DataFrame): The DataFrame to transform.
        data_name (str): data name of the dim
    Returns:
        pd.DataFrame: The transformed DataFrame.
    
    '''
    
    columns_to_rename = {'id': 'transaction_id' , 'use_chip': 'card_entry_method', 'errors':'transaction_error'
                         ,'date':'transaction_date', 'client_id':'user_id'}
    query = 'SELECT merchant_key,merchant_id,merchant_city,merchant_state,zip,mcc FROM financial.dim_merchant'
    merge_on = ['merchant_id','merchant_city','merchant_state', 'zip','mcc']
    columns_to_remove = ['merchant_id', 'merchant_city', 'merchant_state','zip', 'mcc','minute_block']
    bins   = [-1, 15, 30, 45, 59] #use it to create ['minute_block']
    labels = ['0015', '1630', '3145', '4659'] #use it to create ['minute_block']

    try:
        logger.info(f'Starting transform {data_name} fact data')

        #df = df.copy()
        df_cleaned = df.rename(columns=columns_to_rename)
        df_cleaned['amount'] = df_cleaned['amount'].str.replace('$','').astype(float)
        df_cleaned.loc[(df_cleaned['merchant_state'].isna()) & (df_cleaned['merchant_city'] == 'ONLINE'), ['merchant_state']] = 'ONLINE'
        df_cleaned.loc[(df_cleaned['merchant_state'] == 'ONLINE') & (df_cleaned['merchant_city'] == 'ONLINE'), 'zip'] = 0
        df_cleaned['zip'] = df_cleaned['zip'].fillna(0).astype(int)
        #df_cleaned['zip'] = df_cleaned['zip'].astype(str).str.replace(r'\.0$', '', regex=True)

        #create the date_key
        df_cleaned['minute_block'] = pd.cut(pd.to_datetime(df_cleaned['transaction_date']).dt.minute, bins=bins, labels=labels).astype(str)
        df_cleaned['date_key'] = (
            df_cleaned['transaction_date'].astype(str)
                .str.replace(r'[^A-Za-z0-9]', '', regex=True)
                .str[:8]
            + df_cleaned['minute_block'].astype(str)
            )
        
        df_cleaned['date_key'] = df_cleaned['date_key'].astype(int)
        df_cleaned['transaction_date'] = pd.to_datetime(df_cleaned['transaction_date'],cache=True)
        
        engine = load.create_database_connection()
        mcc = pd.read_sql(query,engine)
        df_cleaned = df_cleaned.merge(mcc, left_on= merge_on, right_on= merge_on, how='left',sort=False)
        df_cleaned = df_cleaned.drop(columns= columns_to_remove)

    
        logger.info(f'✅ Successfully transformed {data_name} data with {len(df_cleaned)} records')
        return df_cleaned
        

    except Exception as e:
        logger.info(f'❌Error transforming {data_name} data: {e}')
        raise

In [7]:
dim = transform_fact_transactions_(df,'transactions')

In [8]:
dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954066 entries, 0 to 3954065
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   transaction_id     int64         
 1   transaction_date   datetime64[ns]
 2   user_id            int64         
 3   card_id            int64         
 4   amount             float64       
 5   card_entry_method  object        
 6   transaction_error  object        
 7   date_key           int64         
 8   merchant_key       int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 271.5+ MB


In [10]:
dim.sample(10)

,transaction_id,transaction_date,user_id,card_id,amount,card_entry_method,transaction_error,date_key,merchant_key
1108001,20246263,2017-10-17 07:16:00,1453,1117,1.29,Swipe Transaction,NaN,201710171630,83
2785193,22315478,2018-12-29 13:28:00,169,5717,46.24,Chip Transaction,NaN,201812291630,2785193
1943425,21275315,2018-05-23 11:30:00,1430,4383,12.06,Swipe Transaction,NaN,201805231630,24776
678893,19717093,2017-06-27 07:43:00,1147,2042,33.35,Swipe Transaction,NaN,201706273145,38867
1975477,21314826,2018-05-31 16:38:00,1448,126,23.55,Online Transaction,NaN,201805313145,0
3322654,22981631,2019-05-19 17:15:00,1718,2732,17.30,Chip Transaction,NaN,201905190015,4263
2123307,21497528,2018-07-09 11:39:00,870,3626,361.27,Chip Transaction,NaN,201807093145,148400
437731,19419158,2017-04-25 09:13:00,416,2399,2.06,Chip Transaction,NaN,201704250015,4870
577838,19592348,2017-05-31 18:23:00,1391,5995,47.36,Chip Transaction,NaN,201705311630,903
2104671,21474484,2018-07-04 14:22:00,1115,374,15.45,Chip Transaction,NaN,201807041630,8101


In [16]:
dim['date_key'].astype(str).str.len().value_counts(dropna=False)

date_key
12    3954066
Name: count, dtype: int64

In [ ]:
import pandas as pd
import ETL.config as config
import ETL.utils as utils
import ETL.load_data as load
import ETL.extract_data as extract
import logging


logger = logging.getLogger(__name__)

def transform_fact_transactions(df: pd.DataFrame, data_name: str)->pd.DataFrame: 
    '''
    Function to transform TRANSACTIONS fact data

    Parameters:
        df (pd.DataFrame): The DataFrame to transform.
        data_name (str): data name of the dim
    Returns:
        pd.DataFrame: The transformed DataFrame.
    
    '''
    
    columns_to_rename = {'id': 'transaction_id' , 'use_chip': 'card_entry_method', 'errors':'transaction_error'
                         ,'date':'transaction_date', 'client_id':'user_id'}
    query = 'SELECT merchant_key,merchant_id,merchant_city,merchant_state,zip,mcc FROM financial.dim_merchant'
    merge_on = ['merchant_id','merchant_city','merchant_state', 'zip','mcc']
    columns_to_remove = ['merchant_id', 'merchant_city', 'merchant_state','zip', 'mcc']
    bins   = [-1, 15, 30, 45, 59] #use it to create ['minute_block']
    labels = ['0015', '1630', '3145', '4659'] #use it to create ['minute_block']

    try:
        logger.info(f'Starting transform {data_name} fact data')

        #df = df.copy()
        df_cleaned = df.rename(columns=columns_to_rename)
        df_cleaned['amount'] = df_cleaned['amount'].str.replace('\$','').astype(float)
        df_cleaned.loc[(df_cleaned['merchant_state'].isna()) & (df_cleaned['merchant_city'] == 'ONLINE'), ['merchant_state']] = 'ONLINE'
        df_cleaned.loc[(df_cleaned['merchant_state'] == 'ONLINE') & (df_cleaned['merchant_city'] == 'ONLINE'), 'zip'] = 0
        df_cleaned['zip'] = df_cleaned['zip'].fillna(0).astype(int)
        #df_cleaned['zip'] = df_cleaned['zip'].astype(str).str.replace(r'\.0$', '', regex=True)

        #create the date_key
        # df_cleaned['minute_block'] = pd.cut(pd.to_datetime(df_cleaned['transaction_date']).dt.minute, bins=bins, labels=labels).astype(str)
        # df_cleaned['date_key'] = (
        #     df_cleaned['transaction_date'].astype(str)
        #         .str.replace(r'[^A-Za-z0-9]', '', regex=True)
        #         .str[:10]
        #     + df_cleaned['minute_block'].astype(str)
        #     )
        df_cleaned['transaction_date'] = pd.to_datetime(df_cleaned['transaction_date'],cache=True)
        dt = df_cleaned['transaction_date'].dt
        df_cleaned['date_key'] = (
            dt.year.astype(str) +
            dt.month.astype(str).str.zfill(2) +
            dt.day.astype(str).str.zfill(2) +
            dt.hour.astype(str).str.zfill(2)
        ).astype(int)

        #df_cleaned['transaction_date'] = pd.to_datetime(df_cleaned['transaction_date'],cache=True)
        
        engine = load.create_database_connection()
        mcc = pd.read_sql(query,engine)
        df_cleaned = df_cleaned.merge(mcc, left_on= merge_on, right_on= merge_on, how='left',sort=False)
        df_cleaned = df_cleaned.drop(columns= columns_to_remove)

    
        logger.info(f'✅ Successfully transformed {data_name} data with {len(df_cleaned)} records')
        return df_cleaned
        

    except Exception as e:
        logger.info(f'❌Error transforming {data_name} data: {e}')
        raise

In [49]:
fact = transform_fact_transactions2(df,'transactions')

KeyError: 'year'

In [24]:
df_cleaned = df.copy()

In [42]:
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'],cache=True)
df_cleaned['minute_block'] = (df_cleaned['date'].dt.minute // 15 * 15).astype(str).str.zfill(2)
dt = df_cleaned['date'].dt
df_cleaned['date_key'] = (
    dt.year.astype(str) +
    dt.month.astype(str).str.zfill(2) +
    dt.day.astype(str).str.zfill(2) 
    )


In [43]:
df_cleaned.sample(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,date_key,minute_block
2199024,21590961,2018-07-29 06:14:00,1813,4638,$13.34,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN,20180729,00
3557281,23271490,2019-07-20 06:14:00,1693,2266,$-63.00,Swipe Transaction,59935,Scottsdale,AZ,85260.0,5499,NaN,20190720,00
870273,19953592,2017-08-16 06:41:00,1595,2429,$28.67,Chip Transaction,50783,Bronx,NY,10466.0,5411,NaN,20170816,30
3357226,23024249,2019-05-28 20:52:00,1003,3130,$200.00,Swipe Transaction,27092,Philadelphia,PA,19111.0,4829,NaN,20190528,45
2510744,21976024,2018-10-18 17:02:00,197,1302,$40.14,Chip Transaction,98648,Wilmore,KY,40390.0,5814,NaN,20181018,00
